<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# NeurIPS - Run Skedastic

Goal : Test attentive and conv in other skedastic case

Data : SVHN, CIFAR32 (MNIST Already Ran in Losses)

Models : SelfAttnLNP homoskedastic and ConvLNPXL heteroskedastic

Loss : NLLLloss and CNPF

Runs : 5

In [1]:
import os

os.chdir("..")

import submitit

from train_imgs import main, parse_arguments
from results import get_exp_args

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train = True
        return submitit.utils.DelayedSubmission(self, args)

    def __call__(self, args):
        job_env = submitit.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)

In [24]:
args = get_exp_args("exp_collapse", is_load=False)
len(args)

6

In [25]:
executor=submitit.SlurmExecutor(folder="logs/%j", max_num_timeout=3)
executor.update_parameters(num_gpus=1, 
                           time=60*24,  
                           cpus_per_task=10, 
                           mem='32GB',
                          partition="priority",
                           comment="neurips"
                          )

In [26]:
jobs = executor.map_array(Run(), args)

In [27]:
############################################################

In [33]:
jobs

[SlurmJob<job_id=26762483_0, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26762483_1, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26762483_2, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26762483_3, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26762483_4, task_id=0, state="COMPLETED">,
 SlurmJob<job_id=26762483_5, task_id=0, state="COMPLETED">]

In [21]:
for job in jobs:
    print("--------------------------------")
    print(job.stdout())

--------------------------------
submitit INFO (2020-05-24 11:15:15,501) - Starting with JobEnvironment(job_id=26760695_0, hostname=learnfair0486, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2020-05-24 11:15:15,501) - Loading pickle: /private/home/yannd/projects/NPF/logs/26760695_0/26760695_0_submitted.pkl

--- Training mnist/ConvNPFXL_NllLNPF/run_0 ---

  epoch    train_loss    valid_loss    cp       dur
-------  ------------  ------------  ----  --------
      1     -875.0362    -1089.3674     +  256.9775
      2    -1112.7765    -1209.1669     +  256.4572
submitit WARNING (2020-05-24 11:24:42,234) - Bypassing sigterm.

--------------------------------
submitit INFO (2020-05-24 11:20:54,215) - Starting with JobEnvironment(job_id=26760695_1, hostname=learnfair0229, local_rank=0(1), node=0(1), global_rank=0(1))
submitit INFO (2020-05-24 11:20:54,215) - Loading pickle: /private/home/yannd/projects/NPF/logs/26760695_1/26760695_1_submitted.pkl

--- Training mnist/ConvNPFX

In [11]:
for job in jobs:
    print("--------------------------------")
    print(job.stderr())

--------------------------------

--------------------------------

--------------------------------
/private/home/yannd/projects/NPF/neuralproc/architectures/mlp.py:67: UserWarning: hidden_size=32 smaller than output=128 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))

--------------------------------

--------------------------------

--------------------------------

--------------------------------

--------------------------------

--------------------------------
/private/home/yannd/projects/NPF/neuralproc/architectures/mlp.py:67: UserWarning: hidden_size=32 smaller than output=128 and input=128. Setting it to 128.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))

--------------------------------

--------------------------------

--------------------------------

--------------------------------

--------------------------------

--------------------------------
/private/home

In [19]:
for job in jobs:
    job.cancel()